In [9]:
# import modules
import pandas as pd
from constants import *
from helpers import normalise_word_data, cross_join_dataframes, get_jaccard_sim
import numpy as np
from IPython.display import display
from sklearn import preprocessing

**<span style="color:crimson">1. Naive data linkage without blocking</span>**

In [10]:
# read csv files
google_products = pd.read_csv(GOOGLE_SMALL_PATH)
amazon_products = pd.read_csv(AMAZON_SMALL_PATH)

In [11]:
# normalise the text in the name/title columns of the dataframes
google_products['name'] = normalise_word_data(google_products[['name']].astype(str))
amazon_products['title'] = normalise_word_data(amazon_products[['title']].astype(str))

In [12]:
# join the 2 dataframes
joined = cross_join_dataframes(google_products, amazon_products)
# calculate the scores for the names/titles using the jaccard index
joined['name_score'] =joined.apply(lambda row: get_jaccard_sim(row['name'], row['title']), axis=1)
# calculate the scores for the price similarities
joined['price_score'] = joined.apply(lambda row: (min(row['price_x'], row['price_y'])/max(row['price_x'], row['price_y'])), axis=1)
# calculate the final scores with the price weighted less as there is more chance of duplicates
joined['final_score'] = joined['name_score'] + joined['price_score']/2

In [13]:
# threshold determined through trial and error, only concerned with values above this threshold
THRESHOLD = 0.55
joined = joined[joined['final_score'] > THRESHOLD]
# take only the largest score for each amazonID for comparison
joined = joined.sort_values(by='final_score', ascending=False).drop_duplicates(['idAmazon'])
# create new dataframes for faster calculations
predicted = joined.loc[:, ['idAmazon', 'idGoogleBase']].sort_values(by='idAmazon')
true = pd.read_csv(GOOGLE_AMAZON_SMALL_TRUTH_PATH).sort_values(by='idAmazon')

In [14]:
# create a dataframe of tp values
tp_values = []
for index, row in predicted.iterrows():
    for index2, row2 in true.iterrows():
        # check matches
        if row['idAmazon'] == row2['idAmazon'] and row['idGoogleBase'] == row2['idGoogleBase']:
            tp_values.append((row['idAmazon'], row['idGoogleBase']))

tp_df = pd.DataFrame(tp_values)
tp_df.columns = ['idAmazon', 'idGoogleBase']
display(tp_df.head())

,idAmazon,idGoogleBase
0,1931102953,http://www.google.com/base/feeds/snippets/1272...
1,b00002s6sc,http://www.google.com/base/feeds/snippets/1049...
2,b00004nhn7,http://www.google.com/base/feeds/snippets/1843...
3,b000051sgq,http://www.google.com/base/feeds/snippets/1758...
4,b00006gxbz,http://www.google.com/base/feeds/snippets/9070...


In [15]:
# calculate precision and recall using tp, fp, fn
tp = len(tp_df)
fp = len(predicted) - tp
fn = len(true) - tp - fp
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print(f'precision = {precision}')
print(f'recall = {recall}')

precision = 0.926829268292683
recall = 0.9421487603305785


**<span style="color:green">DISCUSSION**</span><br>
After testing, it was decided that manufacturer and description should not be used in linkage as this lead to too much variance in results. 

A jaccard index, which is used to measure the overlap of two strings, was used to compare the normalized strings in order to get a score based on the similarity of the titles, this was used as titles/names will usually be quite similar across platforms and the jaccard index will usually lead to accurate results while measuring short titles such as the ones in these datasets.

to calculate the similarity of the prices, i took the minimum value of the two and divided that by the maximum value of the two, this leads to creating a (smaller than one) score based on how much smaller the first number is from the second.

The final score was decided by summing the name score and half the price score. The reason half the price score was used was due to it being a less accurate representation of similarity (multiple items can have the same/similar price) in comparison to name which will very rarely have the same/similar values.

The threshold for determining the scores was done through trial and error in order to get the best balance between precision and recall. We also only accounted for the idAmazon's with the highest final scores when comparing with our truth dataset as there can only be one true match for each id, this sufficienty improved the performance of our linkage.

The performance shows us precision ~ 0.927 and recall ~ 0.942. These values both appear to be very good as we have a very high rate of correct linkage between our datasets while still covering a large amount of the true values. 

**<span style="color:crimson">1. Blocking for efficient data linkage</span>**

In [16]:
google_products = pd.read_csv(GOOGLE_PATH)
amazon_products = pd.read_csv(AMAZON_PATH)

In [17]:
google_products['name'] = normalise_word_data(google_products[['name']].astype(str))
amazon_products['title'] = normalise_word_data(amazon_products[['title']].astype(str))

In [61]:
def ngrams(s):
    ngrams = []
    for i in range(len(s)-1):
        ngrams.append(s[i] + s[i+1])
    return ngrams

def score(ser1, ser2):
    done = []
    count = 0
    print(ser1)
    for i in ser1:
        if i in ser2 and i not in done:
            count += 1
    return count


['sp', 'pe', 'el', 'll']

In [66]:
google_products['ngrams_g'] = google_products.apply(lambda row: ngrams(row['name']), axis=1)
google_products['ngrams_length_g'] = google_products.apply(lambda row: len(ngrams(row['name'])), axis=1)
amazon_products['ngrams_a'] = amazon_products.apply(lambda row: ngrams(row['title']), axis=1)
amazon_products['ngrams_length_a'] = amazon_products.apply(lambda row: len(ngrams(row['title'])), axis=1)

In [67]:
max_block_size = max(google_products['ngrams_length_g'].max(), amazon_products['ngrams_length_a'].max())

for i in range(4, max_block_size):
    block_google = google_products[google_products['ngrams_length_g'] == i]
    block_amazon = amazon_products[amazon_products['ngrams_length_a'] == i]
    joined = cross_join_dataframes(block_google, block_amazon)
    display(joined)
    joined['score'] = 2*joined.apply(lambda row: score(row['ngrams_g'], row['ngrams_a']))/(joined['ngrams_length_g'] + joined['ngrams_length_a'])
    
    
    

,id,name,description_x,manufacturer_x,price_x,ngrams_x,ngrams_length_x,ngrams_g,ngrams_length_g,idAmazon,title,description_y,manufacturer_y,price_y,ngrams_y,ngrams_length_y,ngrams_a,ngrams_length_a
0,http://www.google.com/base/feeds/snippets/2096...,spell,all the important spelling skills your child n...,NaN,4.93,"[sp, pe, el, ll]",4,"[sp, pe, el, ll]",4,b00005bigp,shape,NaN,school zone,9.99,"[sh, ha, ap, pe]",4,"[sh, ha, ap, pe]",4
1,http://www.google.com/base/feeds/snippets/1331...,luxor,as addictive as it is exciting luxor provides ...,NaN,12.95,"[lu, ux, xo, or]",4,"[lu, ux, xo, or]",4,b00005bigp,shape,NaN,school zone,9.99,"[sh, ha, ap, pe]",4,"[sh, ha, ap, pe]",4


KeyError: ('ngrams_g', 'occurred at index id')

PART 2

In [ ]:
yeast_data = pd.read_csv(YEAST_PATH)

In [ ]:
yeast_data['Class'] = yeast_data.Class.replace(to_replace=yeast_data.Class.unique(), value=[0, 1])


In [ ]:
from sklearn.impute import SimpleImputer    

In [ ]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')

In [ ]:
X_mean = pd.DataFrame(mean_imputer.fit_transform(yeast_data.drop('Class', axis=1)))
X_median = pd.DataFrame(median_imputer.fit_transform(yeast_data.drop('Class', axis=1)))
Y = yeast_data['Class']

In [ ]:
X_mean.describe()

In [ ]:
X_median.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler, normalize
stdscl = StandardScaler(with_mean=True, with_std=True)

In [ ]:
X_std = pd.DataFrame(stdscl.fit_transform(X_median))
X_norm = pd.DataFrame(normalize(X_median))

In [ ]:
X_std.describe()

In [ ]:
X_norm.describe()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
dtc = DecisionTreeClassifier()

In [ ]:
X_norm['label'] = Y
X_norm = X_norm.sample(frac=1).reset_index(drop=True)

In [ ]:
X_train_with_labels = X_norm[0:1000]
X_test_with_labels = X_norm[1000:]

In [ ]:
Y_train = X_train_with_labels['label']
X_train = X_train_with_labels.drop('label', axis=1)
Y_test = X_test_with_labels['label']
X_test = X_test_with_labels.drop('label', axis=1)

In [ ]:
knn.fit(X_train, Y_train)
y_pred = knn.predict(X_train)

In [ ]:
from sklearn.metrics import precision_score
score = precision_score(Y_train, y_pred)

In [ ]:
score